# Vectorization 
(by Tevfik Aytekin)

Examples showing the speed difference between vectorization vs. for loops in Python.

### Dot Product

In [23]:
# This example is taken from a lecture of Andrew Ng.
import numpy as np
import time

a = np.random.rand(1000000)
b = np.random.rand(1000000)


c = 0
tic = time.time()
for i in range(1000000):
    c+=a[i]*b[i]
toc = time.time()

print(c)
print("For loop: "+str(1000*(toc-tic))+"ms")
tic = time.time()
c = np.dot(a,b)
toc = time.time()

print(c)
print("Vectorized version: "+str(1000*(toc-tic))+"ms")



250399.5057003638
For loop: 609.1539859771729ms
250399.5057003618
Vectorized version: 1.1780261993408203ms


### Matrix Multiplication

In [24]:
Ax = 100
Ay = 100
Bx = 100
By = 100
A = np.random.rand(Ax,Ay)
B = np.random.rand(Bx,By)

result = np.zeros((Ax,  By))
tic = time.time()
for i in range(Ax):
    for j in range(By):
        for k in range(Bx):
            result[i][j] += A[i][k] * B[k][j]
toc = time.time()
print("For loop: "+str(1000*(toc-tic))+"ms")

result = np.zeros((Ax,  By))
tic = time.time()
result = np.dot(Ax,By)
toc = time.time()
print("Vectorized version: "+str(1000*(toc-tic))+"ms")


For loop: 1421.5929508209229ms
Vectorized version: 0.19216537475585938ms


### dot vs matmul

In Python in addition to dot function there is another matrix multiplication function called matmul. Look at this [discussion](https://stackoverflow.com/questions/34142485/difference-between-numpy-dot-and-python-3-5-matrix-multiplication) for their difference.